# Proyek Klasifikasi Gambar: [Input Nama Dataset]
- **Nama:** Salwa Zahrah Dasuki
- **Email:** Salwazd2004@gmail.com
- **ID Dicoding:** MC849D5X0945

## Import Semua Packages/Library yang Digunakan

In [3]:
import os
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing import image

## Data Preparation

### Data Loading

In [4]:
original_dataset_dir = "Dataset"

### Data Preprocessing

#### Split Dataset

In [5]:
base_dir = "Split_Dataset"
train_dir = os.path.join(base_dir, "Train")
val_dir = os.path.join(base_dir, "Validation")
test_dir = os.path.join(base_dir, "Test")

train_ratio, val_ratio, test_ratio = 0.7, 0.15, 0.15

for folder in [train_dir, val_dir, test_dir]:
    os.makedirs(folder, exist_ok=True)

for class_name in os.listdir(original_dataset_dir):
    class_path = os.path.join(original_dataset_dir, class_name)
    if not os.path.isdir(class_path): continue

    images = os.listdir(class_path)
    random.shuffle(images)
    
    n_total = len(images)
    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)

    train_images = images[:n_train]
    val_images = images[n_train:n_train+n_val]
    test_images = images[n_train+n_val:]

    for subset, subset_images in zip([train_dir, val_dir, test_dir], [train_images, val_images, test_images]):
        class_subset_dir = os.path.join(subset, class_name)
        os.makedirs(class_subset_dir, exist_ok=True)
        for img in subset_images:
            shutil.copy2(os.path.join(class_path, img), os.path.join(class_subset_dir, img))

## Modelling

In [ ]:
# 1. Data Generator
img_size = (64, 64) 
batch_size = 32     

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_gen = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_gen = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# 2. CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),  # ✅ Disamakan dengan img_size
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_gen.num_classes, activation='softmax')  # jumlah kelas sesuai dataset
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# 3. Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]

# 4. Training
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=callbacks
)


Found 5607 images belonging to 12 classes.
Found 2657 images belonging to 12 classes.
Found 2717 images belonging to 12 classes.


d:\Data\conda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 12)             │         1,548 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 684,748 (2.61 MB)

...

176/176 ━━━━━━━━━━━━━━━━━━━━ 378s 2s/step - accuracy: 0.6700 - loss: 0.9732 - val_accuracy: 0.7768 - val_loss: 0.6709
Epoch 5/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.6980 - loss: 0.9188 

176/176 ━━━━━━━━━━━━━━━━━━━━ 3731s 21s/step - accuracy: 0.6980 - loss: 0.9186 - val_accuracy: 0.8081 - val_loss: 0.5854
Epoch 6/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 294s 2s/step - accuracy: 0.7244 - loss: 0.8184 - val_accuracy: 0.7746 - val_loss: 0.6349
Epoch 7/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7286 - loss: 0.7805

176/176 ━━━━━━━━━━━━━━━━━━━━ 642s 4s/step - accuracy: 0.7287 - loss: 0.7803 - val_accuracy: 0.8171 - val_loss: 0.5200
Epoch 8/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7629 - loss: 0.6996

176/176 ━━━━━━━━━━━━━━━━━━━━ 818s 5s/step - accuracy: 0.7629 - loss: 0.6996 - val_accuracy: 0.8355 - val_loss: 0.4570
Epoch 9/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7577 - loss: 0.7120

176/176 ━━━━━━━━━━━━━━━━━━━━ 538s 3s/step - accuracy: 0.7577 - loss: 0.7119 - val_accuracy: 0.8393 - val_loss: 0.4503
Epoch 10/10
 47/176 ━━━━━━━━━━━━━━━━━━━━ 7:00 3s/step - accuracy: 0.8059 - loss: 0.5877

## Evaluasi dan Visualisasi

In [ ]:
# Visualisasi akurasi dan loss
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Loss')
plt.legend()

plt.show()

# Evaluasi
loss, acc = model.evaluate(test_gen)
print(f"Test Accuracy: {acc:.4f}")

: 

In [ ]:
print("Final Val Accuracy:", history.history['val_accuracy'][-1])

Final Val Accuracy: 0.7309058904647827


## Konversi Model

In [ ]:
# Simpan SavedModel
model.save("saved_model")

# Konversi ke TF Lite
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model")
tflite_model = converter.convert()
os.makedirs("tflite", exist_ok=True)
with open("tflite/model.tflite", "wb") as f:
    f.write(tflite_model)

# Konversi ke TFJS
!pip install tensorflowjs
!mkdir tfjs_model
!tensorflowjs_converter --input_format=tf_saved_model saved_model tfjs_model

INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


Saved artifact at 'saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 12), dtype=tf.float32, name=None)
Captures:
  2871423065104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2871423066256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2871423062608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2871423064336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2871423065680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2871423065296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2871423066640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2871423065872: TensorSpec(shape=(), dtype=tf.resource, name=None)


## Inference (Optional)

In [ ]:
# Gambar contoh dari test set
img_path = "Split_Dataset/Test/<NamaKelas>/contoh.jpg"  # Ganti <NamaKelas> dan nama file

img = image.load_img(img_path, target_size=(150,150))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

prediction = model.predict(img_tensor)
predicted_class = list(train_gen.class_indices.keys())[np.argmax(prediction)]
print(f"Prediksi: {predicted_class}")

Input shape to model: (1, 64, 64, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Gambar: /Klasifikasi/Dataset\Black-grass\1.png
Prediksi: Loose Silky-bent
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 624ms/step
Prediksi kelas: 6
